# A. **N1**

In this section, various variables that contribute to the performance of each client were analyzed, such as billing, coverage, stability, and portfolio mix. These variables were then weighted and combined to generate a score, which enabled the segmentation of clients into four categories: diamond, gold, silver, and bronze.

# A.1 Data Preparation

In [1]:
import pandas as pd
import numpy as np

# VENTAS
file = "./Data/Facturación.xlsx"
df = pd.read_excel(file, header=0)
df.rename(columns={'Código de Cliente': 'Código Dynamics', 'Precio Total de Venta': 'Total Ventas'}, inplace=True)
df = df.iloc[:, [0, 2, 3, 4, 15, 12, 9, 10]]
df = df.loc[(df['Año Movimiento'] != 2022) | (df['Mes Movimiento'] != 5)]
ventas = df.iloc[:, [0, 1, 2, 3, 4, 5]]

# CLIENTES
file = "./Data/Tipo de negocio.xlsx"
clientes = pd.read_excel(file)
clientes.rename(columns={'COD DINAMICS ': 'Código Dynamics'}, inplace=True)
clientes = clientes.iloc[:, [1, 2, 3, 4, 5, 6]]
clientes = clientes.fillna(0)
clientes = clientes.drop_duplicates()
clientes['NOMBRE CLIENTE '] = clientes['NOMBRE CLIENTE '].str.upper()
clientes_merge = pd.merge(ventas.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])

# SKU
file = "./Data/Portafolio.xlsx"
sku = pd.read_excel(file, sheet_name='SKU ZUMA')
sku = sku.iloc[:, [0, 2, 3]]
sku_merge = pd.merge(clientes_merge, sku.reset_index(), how='inner', on=['Articulo'])

# MERGE FINAL
final_merge = sku_merge[["Código Dynamics", "NOMBRE CLIENTE ", "CANAL ACTUAL", "CANAL MO ", "MERCADO MO ",
                         "ETIQUETA TDN ", "Articulo", "SKU_Categoría", "SKU_Subcategoría", "Marca Nombre",
                         "Cantidad", "Total Ventas"]]
final_merge.head(10)


,Código Dynamics,NOMBRE CLIENTE,CANAL ACTUAL,CANAL MO,MERCADO MO,ETIQUETA TDN,Articulo,SKU_Categoría,SKU_Subcategoría,Marca Nombre,Cantidad,Total Ventas
0,1454,AUTO ACEITES SANTOS,0.0,DETALLE,TRADICIONAL,ACEITERA,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,1.0,19.64
1,1888,MARCO TULIO ALDANA MORALES,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,6.0,117.86
2,243,CERSA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,2.0,39.29
3,243,CERSA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,2.0,39.29
4,243,CERSA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,3.0,61.61
5,607,FEDERICO ADALBERTO FLORES ORELLANA,0.0,DETALLE,TRADICIONAL,FERRETERIA,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,6.0,123.21
6,1984,COMERCIAL ANDRINO,0.0,DETALLE,TRADICIONAL,FERRETERIA,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,3.0,61.61
7,2789,RUDY MELGAR,0.0,DETALLE,TRADICIONAL,CONSUMO PROPIO,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,6.0,123.21
8,2789,RUDY MELGAR,0.0,DETALLE,TRADICIONAL,CONSUMO PROPIO,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,2.0,45.54
9,236,MEDARDO MEJIA AGUILAR,0.0,DETALLE,TRADICIONAL,ACEITERA,3EN1-282ML,QUÍMICOS,FLUIDOS FUNCIONALES,WD-40,6.0,112.50


# A.2 Sells

In [2]:
#Preparacion de tabla
puntaje_ventas = final_merge.groupby(["Código Dynamics","NOMBRE CLIENTE ","ETIQUETA TDN ","MERCADO MO ","CANAL MO "] ).agg({'Total Ventas': 'sum'}).reset_index()

#Funciones
def q1(x):
    return x.quantile(0.25)

def q2(x):
    return x.quantile(0.50)

def q3(x):
    return x.quantile(0.75)

vals = {"Total Ventas": [q1,q2,q3]}

#Tabla con TDN y cuartiles de facturacion
diccionario_facturacion = puntaje_ventas.groupby(["CANAL MO "]).agg(vals).reset_index()
diccionario_facturacion = diccionario_facturacion.set_index(["CANAL MO "])

#Valookup entre diccionario y tabla

puntaje_ventas['q1'] = puntaje_ventas["CANAL MO "].map(diccionario_facturacion['Total Ventas',"q1"])
puntaje_ventas['q2'] = puntaje_ventas["CANAL MO "].map(diccionario_facturacion['Total Ventas',"q2"])
puntaje_ventas['q3'] = puntaje_ventas["CANAL MO "].map(diccionario_facturacion['Total Ventas',"q3"])

#Asignacion de Puntaje
puntaje_ventas.loc[(puntaje_ventas['Total Ventas'] >= puntaje_ventas['q3']), 'puntaje_fact'] = "A"
puntaje_ventas.loc[(puntaje_ventas['Total Ventas'] >= puntaje_ventas['q2']) & (puntaje_ventas['Total Ventas'] < puntaje_ventas['q3']), 'puntaje_fact'] = "B"
puntaje_ventas.loc[(puntaje_ventas['Total Ventas'] >= puntaje_ventas['q1']) & (puntaje_ventas['Total Ventas'] < puntaje_ventas['q2']), 'puntaje_fact'] = "C"
puntaje_ventas.loc[puntaje_ventas['Total Ventas'] <= puntaje_ventas['q1'], 'puntaje_fact'] = "D"


#tabla_puntaje_facturacion = puntaje_ventas
tabla_puntaje_facturacion_1 = puntaje_ventas.set_index("Código Dynamics")
tabla_puntaje_facturacion = tabla_puntaje_facturacion_1

#Export to Excel
tabla_puntaje_facturacion.to_excel("./XLSX_files_output/N1_puntaje_facturacion.xlsx",index=True)
tabla_puntaje_facturacion.head(10)




,NOMBRE CLIENTE,ETIQUETA TDN,MERCADO MO,CANAL MO,Total Ventas,q1,q2,q3,puntaje_fact
Código Dynamics,,,,,,,,,
9,ALFRANSA,MOSTRADOR REPUESTOS MOTOS,TRADICIONAL,DETALLE,2992648.05,7261.34,19184.755,49709.6625,A
18,LUBRICANTES GARCIA II,ACEITERA,TRADICIONAL,DETALLE,2585786.01,7261.34,19184.755,49709.6625,A
19,TOMAS ELEAZAR SONTAY,ACEITERA,TRADICIONAL,DETALLE,2390625.82,7261.34,19184.755,49709.6625,A
23,REPUESTOS Y ACCESORIOS PARA MOTOS BARDALEZ,ACEITERA,TRADICIONAL,DETALLE,1271459.79,7261.34,19184.755,49709.6625,A
29,"TXC, S.A.",ACEITERA,TRADICIONAL,DETALLE,1930068.77,7261.34,19184.755,49709.6625,A
31,REPUESTOS MOLINA,MOSTRADOR REPUESTOS VEHICULOS,TRADICIONAL,DETALLE,1553571.35,7261.34,19184.755,49709.6625,A
41,REPUESTOS LA RUTA,ACEITERA,TRADICIONAL,DETALLE,1280282.52,7261.34,19184.755,49709.6625,A
42,LUIS ARTURO GONZALEZ,ACEITERA,TRADICIONAL,DETALLE,1198000.67,7261.34,19184.755,49709.6625,A
43,IMPORTADORA Y EXPORTADORA MALDONADO,ACEITERA,TRADICIONAL,DETALLE,1292466.84,7261.34,19184.755,49709.6625,A


# A.3 Stability

In [3]:
#Preparacion de Tablas
file = "./Data/Facturación.xlsx"
read = pd.read_excel(file, header = 0)
df=read.rename(columns = {'Código de Cliente':'Código Dynamics', 'Precio Total de Venta':'Total Ventas'}, inplace = True)
df=read.iloc[:,[0,2,3,4,15,12,9,10]]
df=df.loc[(df['Año Movimiento'] != 2022) | (df['Mes Movimiento'] != 5) ]
df=df.iloc[:,[2,0,4,5,6]]

codigos_merge = pd.merge(clientes.reset_index(), df.reset_index(), how='inner', on=['Código Dynamics'])
codigos_merge_ = codigos_merge.set_index('Código Dynamics')
codigos_merge_ = codigos_merge_.reset_index(level=0)
codigos_merge_3 = codigos_merge_[["Código Dynamics", "Total Ventas", "Mes Movimiento"]]

#Pivoteo de meses
codigos_merge_pivot = pd.pivot_table(codigos_merge_3, values = 'Total Ventas', index=['Código Dynamics'], columns = 'Mes Movimiento')
codigos_merge_pivot_4 = codigos_merge_pivot.iloc[: , -6:]

#Funciones
def normalizadorx(x):
    if x >0:
        return 1
    else:
        return 0
    
#Calculo de score
codigos_merge_pivot_5=codigos_merge_pivot_4.applymap(normalizadorx)
codigos_merge_pivot_5["score"] = codigos_merge_pivot_5.apply(lambda row: np.sum(row), axis=1) 
codigos_merge_pivot_5 =codigos_merge_pivot_5.reset_index(level=0)
codigos_merge_pivot_5= pd.merge(codigos_merge_pivot_5.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
codigos_merge_pivot_5 = codigos_merge_pivot_5[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","score"]]


#Ventas Promedio Mes
codigos_merge_pivot_4["Venta_Promedio_Mes"] = codigos_merge_pivot_4.apply(lambda row: np.mean(row), axis=1)
codigos_merge_pivot_6 = (codigos_merge_pivot_4.iloc[:,[6]]).fillna(0).reset_index()
codigos_merge_pivot_6 = pd.merge(codigos_merge_pivot_6.reset_index(), codigos_merge_pivot_5.reset_index(), how='inner', on=['Código Dynamics'])
codigos_merge_pivot_7 = codigos_merge_pivot_6[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","score","Venta_Promedio_Mes"]].fillna(0)

# MASK 1----------------------------------------------------------------------------------------------------
codigos_merge_pivot_7mask_1 =codigos_merge_pivot_7

#Creacion de Diccionario con valores de facturacion promedio del canal durante el ultimo semestre
diccionario_ventas = codigos_merge_pivot_7.groupby(["CANAL MO "]).agg({'Venta_Promedio_Mes': 'mean'}).reset_index()
diccionario_ventas = diccionario_ventas.set_index("CANAL MO ")
diccionario_ventas= diccionario_ventas.to_dict()

#Venta promedio del Canal en el ultimo semestre
codigos_merge_pivot_7mask_1['Promedio_Canal'] = codigos_merge_pivot_7mask_1["CANAL MO "].map(diccionario_ventas['Venta_Promedio_Mes'])

#Asignacion de Puntaje
codigos_merge_pivot_7mask_1.loc[(codigos_merge_pivot_7mask_1['score'] >=4) , 'puntaje_est'] = 'B'
codigos_merge_pivot_7mask_1.loc[(codigos_merge_pivot_7mask_1['score'] <4) & (codigos_merge_pivot_7mask_1["score"] >=1 ), 'puntaje_est'] = 'C'
codigos_merge_pivot_7mask_1.loc[(codigos_merge_pivot_7mask_1['score'] ==0), 'puntaje_est'] = 'D'
codigos_merge_pivot_7mask_1.loc[(codigos_merge_pivot_7mask_1['score'] >=5) & (codigos_merge_pivot_7mask_1["Venta_Promedio_Mes"] > codigos_merge_pivot_7mask_1["Promedio_Canal"]), 'puntaje_est'] = 'A'

codigos_merge_pivot_7mask_1 = codigos_merge_pivot_7mask_1.set_index('Código Dynamics')
codigos_merge_pivot_7 = codigos_merge_pivot_7mask_1

#Export to Excel
codigos_merge_pivot_7.to_excel("./XLSX_files_output/N1_puntaje_estabilidad.xlsx",index=True)
codigos_merge_pivot_7.head(10)


C:\Users\DELL\AppData\Local\Temp\ipykernel_1444\1189633406.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codigos_merge_pivot_4["Venta_Promedio_Mes"] = codigos_merge_pivot_4.apply(lambda row: np.mean(row), axis=1)


,NOMBRE CLIENTE,CANAL ACTUAL,MERCADO MO,CANAL MO,ETIQUETA TDN,score,Venta_Promedio_Mes,Promedio_Canal,puntaje_est
Código Dynamics,,,,,,,,,
9,ALFRANSA,0.0,TRADICIONAL,DETALLE,MOSTRADOR REPUESTOS MOTOS,4,83166.119006,1019.533121,B
18,LUBRICANTES GARCIA II,0.0,TRADICIONAL,DETALLE,ACEITERA,6,5514.908360,1019.533121,A
19,TOMAS ELEAZAR SONTAY,0.0,TRADICIONAL,DETALLE,ACEITERA,6,5591.276301,1019.533121,A
23,REPUESTOS Y ACCESORIOS PARA MOTOS BARDALEZ,0.0,TRADICIONAL,DETALLE,ACEITERA,5,21913.755819,1019.533121,A
29,"TXC, S.A.",0.0,TRADICIONAL,DETALLE,ACEITERA,6,2435.474548,1019.533121,A
31,REPUESTOS MOLINA,0.0,TRADICIONAL,DETALLE,MOSTRADOR REPUESTOS VEHICULOS,6,3383.662759,1019.533121,A
41,REPUESTOS LA RUTA,0.0,TRADICIONAL,DETALLE,ACEITERA,6,8438.216150,1019.533121,A
42,LUIS ARTURO GONZALEZ,0.0,TRADICIONAL,DETALLE,ACEITERA,6,7087.890482,1019.533121,A
43,IMPORTADORA Y EXPORTADORA MALDONADO,0.0,TRADICIONAL,DETALLE,ACEITERA,6,20410.007833,1019.533121,A


# A.4 SKU Porfolio

In [4]:
#Agrupacion por Ventas
puntaje_mix = final_merge.groupby(["Código Dynamics","ETIQUETA TDN ","CANAL MO ","NOMBRE CLIENTE ","SKU_Categoría"] ).agg({'Total Ventas': 'sum'})

#Pivot
puntaje_mix_pivot = pd.pivot_table(puntaje_mix, values = 'Total Ventas', index=['Código Dynamics','ETIQUETA TDN ',"CANAL MO ", "NOMBRE CLIENTE "], columns = 'SKU_Categoría').reset_index()

#Defincion de lista Categorias de SKU
lista = final_merge["SKU_Categoría"].tolist()
lista = list(dict.fromkeys(lista))

categoria= puntaje_mix_pivot['ETIQUETA TDN '].tolist()
categoria = list(dict.fromkeys(categoria))

#Funciones
def cumsumx(x):
    porcentaje = (x/x.sum())*100
    ordenado= porcentaje.sort_values(ascending=False,by=lista)
    cumsum= ordenado.cumsum()
    return cumsum

def scorex(x):
    if  x==0:
        return 0
    elif  x>0 and x<=20:
        return 5
    elif x>20 and x<=50:
        return 4
    elif x>50 and x<=80:
        return 3
    elif x>80 and x<=95:
        return 2
    elif x>95 and x<=100:
        return 1
    else:
        return 0
    
def puntajex(x):
    if x<0.5:
        return "D"
    elif x>=0.5 and x<1.5:
        return "C"
    elif x>=1.5 and x<2.5:
        return "B"
    elif x>=2.5:
        return "A"
    
#Calculos
puntaje_mix_pivot_mask = puntaje_mix_pivot["CANAL MO "]== "DETALLE"
puntaje_mix_pivot_filter = puntaje_mix_pivot[puntaje_mix_pivot_mask]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score["puntaje_mix_2"] =score["score"].apply(puntajex)
score = score.reset_index(level=0)
detalle = score


#Joint con Clientes
tabla_puntaje_mix = detalle
tabla_puntaje_mix = pd.merge(tabla_puntaje_mix.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
tabla_puntaje_mix = tabla_puntaje_mix[["Código Dynamics", "NOMBRE CLIENTE ","CANAL ACTUAL","CANAL MO ", "MERCADO MO ","ETIQUETA TDN ","score","puntaje_mix_2"]]

promedio_ventas = final_merge.groupby(["Código Dynamics","NOMBRE CLIENTE ","ETIQUETA TDN ","MERCADO MO ","CANAL MO "] ).agg({'Total Ventas': 'mean'})
tabla_puntaje_mix =pd.merge(tabla_puntaje_mix, codigos_merge_pivot_7[['Promedio_Canal']], on=['Código Dynamics'])
tabla_puntaje_mix =pd.merge(tabla_puntaje_mix, promedio_ventas[["Total Ventas"]], on=['Código Dynamics'])


#Asignacion de Puntaje
tabla_puntaje_mix.loc[(tabla_puntaje_mix['Total Ventas'] >= tabla_puntaje_mix['Promedio_Canal']) & (tabla_puntaje_mix['puntaje_mix_2'] == "A"), 'puntaje_mix'] = "A"
tabla_puntaje_mix.loc[(tabla_puntaje_mix['Total Ventas'] < tabla_puntaje_mix['Promedio_Canal']) & (tabla_puntaje_mix['puntaje_mix_2'] == "A"), 'puntaje_mix'] = "B"
tabla_puntaje_mix.loc[(tabla_puntaje_mix['puntaje_mix_2'] == "B"), 'puntaje_mix'] = "B"
tabla_puntaje_mix.loc[(tabla_puntaje_mix['puntaje_mix_2'] == "C"), 'puntaje_mix'] = "C"
tabla_puntaje_mix.loc[(tabla_puntaje_mix['puntaje_mix_2'] == "D"), 'puntaje_mix'] = "D"

tabla_puntaje_mix = tabla_puntaje_mix[["Código Dynamics", "NOMBRE CLIENTE ","CANAL ACTUAL","CANAL MO ", "MERCADO MO ","ETIQUETA TDN ","score","puntaje_mix"]]


#Export to Excel
tabla_puntaje_mix.to_excel("./XLSX_files_output/N1_puntaje_mix.xlsx",index=True)
tabla_puntaje_mix.head(10)


,Código Dynamics,NOMBRE CLIENTE,CANAL ACTUAL,CANAL MO,MERCADO MO,ETIQUETA TDN,score,puntaje_mix
0,226,GT TRAIDING,0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,1.0,C
1,1413,MOVIMIENTO DE MAQUINARIA COPROPIEDAD,0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,1.5,B
2,18,LUBRICANTES GARCIA II,0.0,DETALLE,TRADICIONAL,ACEITERA,2.5,A
3,1106,INTERSUM S.A.,0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,1.5,B
4,1207,"DISTRIBUIDORA AP&P, SOCIEDAD ANONIMA",0.0,DETALLE,TRADICIONAL,FERRETERIA,1.5,B
5,31,REPUESTOS MOLINA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,3.0,A
6,176,REPUESTOS KAIROS,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,4.0,A
7,90,REPUESTOS DE LEON,0.0,DETALLE,TRADICIONAL,ACEITERA,4.5,A
8,29,"TXC, S.A.",0.0,DETALLE,TRADICIONAL,ACEITERA,3.5,A
9,118,REPUESTOS AUOTOTOTAL,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,3.5,A


# A.5 Coverage

In [5]:
#Importacion de tablas y pivot
puntaje_cobertura = final_merge.groupby(["Código Dynamics","ETIQUETA TDN ","NOMBRE CLIENTE ","SKU_Subcategoría"] ).agg({'Total Ventas': 'count'})
puntaje_cobertura_pivot = pd.pivot_table(puntaje_cobertura, values = 'Total Ventas', index=['Código Dynamics','ETIQUETA TDN ', "NOMBRE CLIENTE "], columns = 'SKU_Subcategoría').reset_index()
puntaje_cobertura_pivot= puntaje_cobertura_pivot.fillna(0)

#Definicion de Lista Subcategoria
subcategorias = final_merge["SKU_Subcategoría"].tolist()
subcategorias = list(dict.fromkeys(subcategorias))

#Definicion de diccionario con ponderaciones por subcategoria
diccionario = {"SILICONES" : 3,
                 'LIMPIADORES': 4,
                 'FLUIDOS FUNCIONALES':4,
                 'TRANSMISIÓN':8,
                 'SEMISINTÉTICO':8,
                 'ALTO PERFORMANCE':6,
                 'LÍNEA CONVENCIONAL':6,
                 'SINTÉTICO':8,
                 'REFRIGERANTE':4,
                 'MINERAL':8,
                 'BUJÍAS':1,
                 'FRENOS':3,
                 'EPÓXICOS':1,
                 'CINTAS ADHESIVAS':1,
                 'LIJAS':0,
                 'OTROS':6,
                 'PLUMILLA':0,
                 'ADITIVOS':2,
                 'SUSPENSIÓN':2,
                 'FILTROS':2,
                 'RODAMIENTOS':1,
                 'FAJAS':1,
                 'PINTURA':2,
                 'BOMBILLAS':0,
                 'GRASAS':0}

#Definicion de funciones 
def normalizadorx(x):
    if x >0:
        return 1
    else:
        return 0

def puntajex(x):
    if x<30:
        return "D"
    elif x>=30 and x<50:
        return "C"
    elif x>=50 and x<60:
        return "B"
    elif x>=60:
        return "A"
    
#Aplicacion de funciones    
cobertura = puntaje_cobertura_pivot.set_index('Código Dynamics')
cobertura = cobertura[subcategorias]
cobertura = cobertura.applymap(normalizadorx)
cobertura = cobertura.replace(1, diccionario)
cobertura["score"] = cobertura.apply(lambda row: np.sum(row), axis=1) 
cobertura["puntaje_cobertura"] =cobertura["score"].apply(puntajex)
cobertura = cobertura.reset_index(level=0)
cobertura = pd.merge(cobertura.reset_index(), clientes.reset_index(), how='left', on=['Código Dynamics'])
cobertura = cobertura[["Código Dynamics", "NOMBRE CLIENTE ","CANAL ACTUAL","CANAL MO ", "MERCADO MO ","ETIQUETA TDN ","score","puntaje_cobertura"]].set_index('Código Dynamics')

#Export to Excel
cobertura.to_excel("./XLSX_files_output/N1_puntaje_cobertura.xlsx",index=True)
cobertura.head(10)


,NOMBRE CLIENTE,CANAL ACTUAL,CANAL MO,MERCADO MO,ETIQUETA TDN,score,puntaje_cobertura
Código Dynamics,,,,,,,
9,ALFRANSA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS MOTOS,43,C
18,LUBRICANTES GARCIA II,0.0,DETALLE,TRADICIONAL,ACEITERA,60,A
19,TOMAS ELEAZAR SONTAY,0.0,DETALLE,TRADICIONAL,ACEITERA,55,B
23,REPUESTOS Y ACCESORIOS PARA MOTOS BARDALEZ,0.0,DETALLE,TRADICIONAL,ACEITERA,57,B
29,"TXC, S.A.",0.0,DETALLE,TRADICIONAL,ACEITERA,64,A
31,REPUESTOS MOLINA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,59,B
41,REPUESTOS LA RUTA,0.0,DETALLE,TRADICIONAL,ACEITERA,51,B
42,LUIS ARTURO GONZALEZ,0.0,DETALLE,TRADICIONAL,ACEITERA,50,B
43,IMPORTADORA Y EXPORTADORA MALDONADO,0.0,DETALLE,TRADICIONAL,ACEITERA,52,B


# A.6 N1_Scoring

In [6]:
#Agregacion de puntajes
consolidado_N1 =pd.merge(tabla_puntaje_mix, tabla_puntaje_facturacion[['puntaje_fact']], on=['Código Dynamics'])
consolidado_N1 =pd.merge(consolidado_N1, cobertura[['puntaje_cobertura']], on=['Código Dynamics'])
consolidado_N1 =pd.merge(consolidado_N1, codigos_merge_pivot_7[['puntaje_est']], on=['Código Dynamics'])
consolidado_N1 = consolidado_N1.loc[:, consolidado_N1.columns != 'score'].set_index('Código Dynamics')

#Funciones
def normalizadorx(x):
    if x == "A":
        return 4
    if x == "B":
        return 3
    if x == "C":
        return 2
    if x == "D":
        return 1
    else:
        return x


#Definicion de ponderaciones y creacion de diccionario    
diccionario = {"puntaje_mix" : 0.30,
                 'puntaje_fact':0.45 ,
                 'puntaje_cobertura':0.10,
                 'puntaje_est':0.10,
                 }    
    
#Aplicacion de funciones
consolidado_2 = consolidado_N1.applymap(normalizadorx)


#Aplicacion de ponderaciones
for key, value in diccionario.items(): 
    consolidado_2[key] = consolidado_2[key] * value

#Suma de scores ponderados    
consolidado_2['score_final'] =round(consolidado_2.sum(axis=1))

#Aplicacion de condicionales para puntaje final
consolidado_2.loc[(consolidado_2['score_final'] ==1 ), 'puntaje_final'] = "BRONCE"
consolidado_2.loc[(consolidado_2['score_final'] ==2 ), 'puntaje_final'] = "PLATA"
consolidado_2.loc[(consolidado_2['score_final'] ==3 ), 'puntaje_final'] = "ORO"
consolidado_2.loc[(consolidado_2['score_final'] ==4 ), 'puntaje_final'] = "DIAMANTE"

#Reordenamiento de DataFrame
consolidado_final =pd.merge(consolidado_2, consolidado_N1[['puntaje_fact',"puntaje_mix","puntaje_cobertura","puntaje_est"]], on=['Código Dynamics']).reset_index()
consolidado_final = consolidado_final.iloc[:,[0,1,2,3,4,5,6,13,7,12,8,14,9,15,10,11]]
consolidado_final = consolidado_final.drop_duplicates()

consolidado_final_0= consolidado_final.reset_index()
consolidado_final_0 = consolidado_final_0.astype({'Código Dynamics':'string'})
consolidado_final_0['Código Dynamics'] = consolidado_final_0['Código Dynamics'].apply(lambda x: x.zfill(6))

#Export to Excel
consolidado_final_0.to_excel("./XLSX_files_output/N1_PUNTAJE_FINAL.xlsx",index=False)
consolidado_final_0.head(10)





C:\Users\DELL\AppData\Local\Temp\ipykernel_1444\3685375997.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  consolidado_2['score_final'] =round(consolidado_2.sum(axis=1))


,index,Código Dynamics,NOMBRE CLIENTE,CANAL ACTUAL,CANAL MO,MERCADO MO,ETIQUETA TDN,puntaje_mix_x,puntaje_mix_y,puntaje_fact_x,puntaje_fact_y,puntaje_cobertura_x,puntaje_cobertura_y,puntaje_est_x,puntaje_est_y,score_final,puntaje_final
0,0,000226,GT TRAIDING,0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,0.6,C,1.8,A,0.1,D,0.4,A,3.0,ORO
1,1,001413,MOVIMIENTO DE MAQUINARIA COPROPIEDAD,0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,0.9,B,1.8,A,0.1,D,0.2,C,3.0,ORO
2,2,000018,LUBRICANTES GARCIA II,0.0,DETALLE,TRADICIONAL,ACEITERA,1.2,A,1.8,A,0.4,A,0.4,A,4.0,DIAMANTE
3,3,001106,INTERSUM S.A.,0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,0.9,B,1.8,A,0.1,D,0.2,C,3.0,ORO
4,4,001207,"DISTRIBUIDORA AP&P, SOCIEDAD ANONIMA",0.0,DETALLE,TRADICIONAL,FERRETERIA,0.9,B,1.8,A,0.1,D,0.3,B,3.0,ORO
5,5,000031,REPUESTOS MOLINA,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,1.2,A,1.8,A,0.3,B,0.4,A,4.0,DIAMANTE
6,6,000176,REPUESTOS KAIROS,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,1.2,A,1.8,A,0.4,A,0.4,A,4.0,DIAMANTE
7,7,000090,REPUESTOS DE LEON,0.0,DETALLE,TRADICIONAL,ACEITERA,1.2,A,1.8,A,0.4,A,0.4,A,4.0,DIAMANTE
8,8,000029,"TXC, S.A.",0.0,DETALLE,TRADICIONAL,ACEITERA,1.2,A,1.8,A,0.4,A,0.4,A,4.0,DIAMANTE
9,9,000118,REPUESTOS AUOTOTOTAL,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,1.2,A,1.8,A,0.3,B,0.4,A,4.0,DIAMANTE


# B. **N2**

In this section, the potential of each client was analyzed based on parameters such as company size, number of employees, and infrastructure. These variables were used to generate a score that determines the potential of each client, categorized as high, medium, or low.

# B.1 Data Preparation

In [7]:
import pandas as pd
import numpy as np

#CLIENTES - TIPO DE NEGOCIO!
file = "./Data/Tipo de Negocio.xlsx"
read = pd.read_excel(file)
clientes = read.rename(columns = {'COD DINAMICS ':'Código Dynamics'}, inplace = True)
clientes = read.iloc[:,[1,2,3,4,5,6]]
clientes = clientes.fillna(0)
clientes = clientes.drop_duplicates()
clientes['NOMBRE CLIENTE '] = clientes['NOMBRE CLIENTE '].str.upper()

# POTENCIAL DE NEGOCIO - ENCUESTA!
file = "./Data/Potencial de Negocio.xlsx"
read = pd.read_excel(file, header = 1, sheet_name='CENSO ')
tabla = read.rename(columns = {'CODIGO DE CLIENTE PDV':'Código Dynamics'}, inplace = True)
tabla = read.astype({'Código Dynamics':'int64'})
tabla= tabla.dropna(subset=['Código Dynamics'])
tabla = tabla.drop_duplicates(subset=['Código Dynamics']).set_index("Código Dynamics")


# B.2 Power

In [8]:
#CALCULO DE POTENCIA DEL NEGOCIO
potencia = tabla[["1.00 VENTA  LITROS DE ACEITE DE MOTOS ",
                  "2.00 VENTA LITROS DE ACEITE DE VEHICULOS ", 
                  "3.00 VENTA LITROS DE ACEITE DISEL ", 
                  "4.001 VENTA  LITROS DE LIQUIDO DE FRENOS ",
                  "4.002 VENTA LITROS DE POWER STEERING ",
                  "4.003 VENTA LITROS DE REFRIGERANTE ",
                  "4.004 VENTA  LITROS DE LIMPIADORES ",
                  "5.001 VENTA UNIDADES DE PASTILLAS DE FRENOS ",
                  "5.002 VENTA UNIDADES DE FAJAS VENDE", 
                  "5.003 VENTA UNIDADES DE AMORTIGUADORES ", 
                  "5.004 VENTA UNIDADES DE RODAMIENTOS ",
                  "6.00 VENTA UNIDADES  PLUMILLAS, BOMBILLOS Y/O LIJAS "]]

potencia = potencia.replace('-',np.nan ,regex=True)
potencia = potencia.replace('9999',np.nan ,regex=True)
potencia = potencia.replace('999',np.nan ,regex=True).fillna(0).astype('int32')
potencia["potencia_total"] = potencia.apply(lambda row: np.sum(row), axis=1).astype('int32')

#CALCULO DE CUARTILES
potencia["q1"] =potencia["potencia_total"].quantile(0.30)
potencia["q2"] =potencia["potencia_total"].quantile(0.70)

#ASIGNACION DE PUNTAJES
potencia.loc[(potencia['potencia_total'] < potencia['q1']), 'puntaje_potencia'] = "C"
potencia.loc[(potencia['potencia_total'] >= potencia['q1']) & (potencia['potencia_total'] < potencia['q2']), 'puntaje_potencia'] = "B"
potencia.loc[(potencia['potencia_total'] >= potencia['q2']), 'puntaje_potencia'] = "A"

#MERGE CON CLIENTES
potencia_merge = pd.merge(potencia.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
potencia_merge_ =  potencia_merge[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","potencia_total","q1","q2","puntaje_potencia"]].set_index('Código Dynamics')

#Export to Excel
potencia_merge_.to_excel('./XLSX_files_output/N2_puntaje_potencia.xlsx',index=True)
potencia_merge_.head(10)


,NOMBRE CLIENTE,CANAL ACTUAL,MERCADO MO,CANAL MO,ETIQUETA TDN,potencia_total,q1,q2,puntaje_potencia
Código Dynamics,,,,,,,,,
9419,PEDRO CRUZ,0.0,TRADICIONAL,DETALLE,MOSTRADOR REPUESTOS MOTOS,30,33.0,110.0,C
9416,DAYANA PAOLA PINEDA ZAMORA,0.0,TRADICIONAL,DETALLE,ACEITERA,98,33.0,110.0,B
9415,JOEL ALFREDO DE LEON TAQUE,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0,33.0,110.0,C
9408,ELFIDO SALAZAR DE LA CRUZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,72,33.0,110.0,B
9406,ESTUARDO TECU GARCIA,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,32,33.0,110.0,C
9405,JANS KENNY LÓPEZ NORIEGA,0.0,TRADICIONAL,DETALLE,CONSUMO PROPIO,12,33.0,110.0,C
9398,"PESQUEROS DE GUATEMALA, S.A.",0.0,TRADICIONAL,DETALLE,SERVICIOS VARIOS,10,33.0,110.0,C
9394,VICTOR EMILIO IXCAJOC TICUN,0.0,TRADICIONAL,DETALLE,TALLER DE VEHICULO,60,33.0,110.0,B
9387,WILSON FRANKYN PEREZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,18,33.0,110.0,C


# B.4 Inversion

In [9]:
tabla_2 = read.drop_duplicates(subset=['Código Dynamics']).set_index("Código Dynamics")
inversion = tabla_2[["10.01 FACHADA",
                  "10.02CENEFAS", 
                  "10.03 RACKS", 
                  "10.04 EXHIBIDORES DE MOSTRADOR",
                  "10.05 EXHIBIDOR DE PISO",
                  "10.06 AFICHES",
                  "10.07 PORTA AFICHE",
                  "10.08 PRECIFICADORES",
                  "10.09 BOLETINES", 
                  "10.10 BANDEROLAS", 
                  "5.004 VENTA UNIDADES DE RODAMIENTOS ",
                  "10.11 MANTA VINILICA",
                  "10.12  MOSTRADOR(ES)"]]


diccionario = {"10.01 FACHADA" : 5,
               "10.02CENEFAS":1,
               "10.03 RACKS":10,
               "10.04 EXHIBIDORES DE MOSTRADOR":10,
               "10.05 EXHIBIDOR DE PISO":10,
               "10.06 AFICHES":5,
               "10.07 PORTA AFICHE":5,
               "10.08 PRECIFICADORES":1,
               "10.09 BOLETINES":1,
               "10.10 BANDEROLAS":5,
               "10.11 MANTA VINILICA":5,
               "10.12  MOSTRADOR(ES)":10}



def normalizadorx(x):
    if x >0:
        return 1
    else:
        return 0

def puntajex(x):
    if x<25:
        return "C"
    elif x>=25 and x<45:
        return "B"
    elif x>=45:
        return "A"

inversion = inversion.replace('-',np.nan ,regex=True).fillna(0).astype('int32')
inversion = inversion.applymap(normalizadorx)
inversion = inversion.replace(1, diccionario)
inversion["score"] = inversion.apply(lambda row: np.sum(row), axis=1) 
inversion["puntaje_inversion"] =inversion["score"].apply(puntajex)

#MERGE CON CLIENTES
inversion_merge = pd.merge(inversion.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
inversion_merge =  inversion_merge[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","score","puntaje_inversion"]].set_index('Código Dynamics')

#Export to Excel
inversion_merge.to_excel('./XLSX_files_output/N2_puntaje_inversion.xlsx',index=True)

inversion_merge.head(10)


,NOMBRE CLIENTE,CANAL ACTUAL,MERCADO MO,CANAL MO,ETIQUETA TDN,score,puntaje_inversion
Código Dynamics,,,,,,,
9419,PEDRO CRUZ,0.0,TRADICIONAL,DETALLE,MOSTRADOR REPUESTOS MOTOS,0,C
9416,DAYANA PAOLA PINEDA ZAMORA,0.0,TRADICIONAL,DETALLE,ACEITERA,31,B
9415,JOEL ALFREDO DE LEON TAQUE,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0,C
9408,ELFIDO SALAZAR DE LA CRUZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,16,C
9406,ESTUARDO TECU GARCIA,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,15,C
9405,JANS KENNY LÓPEZ NORIEGA,0.0,TRADICIONAL,DETALLE,CONSUMO PROPIO,0,C
9398,"PESQUEROS DE GUATEMALA, S.A.",0.0,TRADICIONAL,DETALLE,SERVICIOS VARIOS,15,C
9394,VICTOR EMILIO IXCAJOC TICUN,0.0,TRADICIONAL,DETALLE,TALLER DE VEHICULO,0,C
9387,WILSON FRANKYN PEREZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,50,A


# B.4 Categorization 

In [10]:
pivot = potencia_merge.loc[:, ~potencia_merge.columns.isin(['potencia_total', 'q1', "q2","puntaje_potencia","index"])]

categoria= pivot['ETIQUETA TDN '].tolist()
categoria = list(dict.fromkeys(categoria))

lista =       ["1.00 VENTA  LITROS DE ACEITE DE MOTOS ",
                  "2.00 VENTA LITROS DE ACEITE DE VEHICULOS ", 
                  "3.00 VENTA LITROS DE ACEITE DISEL ", 
                  "4.001 VENTA  LITROS DE LIQUIDO DE FRENOS ",
                  "4.002 VENTA LITROS DE POWER STEERING ",
                  "4.003 VENTA LITROS DE REFRIGERANTE ",
                  "4.004 VENTA  LITROS DE LIMPIADORES ",
                  "5.001 VENTA UNIDADES DE PASTILLAS DE FRENOS ",
                  "5.002 VENTA UNIDADES DE FAJAS VENDE", 
                  "5.003 VENTA UNIDADES DE AMORTIGUADORES ", 
                  "5.004 VENTA UNIDADES DE RODAMIENTOS ",
                  "6.00 VENTA UNIDADES  PLUMILLAS, BOMBILLOS Y/O LIJAS "]


#Funciones
def cumsumx(x):
    porcentaje = (x/x.sum())*100
    ordenado= porcentaje.sort_values(ascending=False,by=lista)
    cumsum= ordenado.cumsum()
    return cumsum

def scorex(x):
    if  x==0:
        return 0
    elif  x>0 and x<=25:
        return 5
    elif x>25 and x<=50:
        return 4
    elif x>50 and x<=75:
        return 3
    elif x>75 and x<=80:
        return 2
    elif x>80 and x<=100:
        return 1
    else:
        return 0
    
def puntajex(x):
    if x<2:
        return "C"
    elif x>=2 and x<4:
        return "B"
    elif x>=4:
        return "A"

puntaje_mix_pivot_mask = pivot["ETIQUETA TDN "]== "ACEITERA"
puntaje_mix_pivot_filter = pivot[puntaje_mix_pivot_mask]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score = score.reset_index(level=0)
aceitera = score

puntaje_mix_pivot_mask = pivot["ETIQUETA TDN "]== "TALLER DE MOTOS"
puntaje_mix_pivot_filter = pivot[puntaje_mix_pivot_mask]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score = score.reset_index(level=0)
taller = score

puntaje_mix_pivot_mask = pivot["ETIQUETA TDN "]== "TALLER DE VEHICULO"
puntaje_mix_pivot_filter = pivot[puntaje_mix_pivot_mask]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score = score.reset_index(level=0)
vehiculo = score

puntaje_mix_pivot_mask = pivot["ETIQUETA TDN "]== "MOSTRADOR REPUESTOS MOTOS"
puntaje_mix_pivot_filter = pivot[puntaje_mix_pivot_mask]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score = score.reset_index(level=0)
moto_repuestos = score

puntaje_mix_pivot_mask = pivot["ETIQUETA TDN "]== "MOSTRADOR REPUESTOS VEHICULOS"
puntaje_mix_pivot_filter = pivot[puntaje_mix_pivot_mask]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score = score.reset_index(level=0)
vehiculo_repuestos = score

array = ['FERRETERIA', 'AGROCENTRO', "GASOLINERA","SERVICIOS VARIOS", "CONSUMO PROPIO",'PINCHAZO','TIENDA DE BARRIO','SUPERMERCADO',"CARWASH"]
puntaje_mix_pivot_filter =pivot.loc[pivot['ETIQUETA TDN '].isin(array)]
puntaje_mix_pivot_filter=puntaje_mix_pivot_filter.set_index('Código Dynamics')
cumsum = cumsumx(puntaje_mix_pivot_filter[lista])
score = cumsum.applymap(scorex)
score["score"] = score.replace('', np.nan).apply(lambda row: np.nanmean(row), axis=1) 
score = score.reset_index(level=0)
otros = score


#AGRUPACION DE TABLAS Y MERGE CON CLIENTES
tabla_puntaje_mix = pd.concat([aceitera,taller,vehiculo,moto_repuestos,vehiculo_repuestos,otros])
tabla_puntaje_mix = pd.merge(tabla_puntaje_mix.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
tabla_puntaje_mix = tabla_puntaje_mix[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","score"]].set_index('Código Dynamics')
tabla_puntaje_mix["puntaje_mix_2"] =tabla_puntaje_mix["score"].apply(puntajex)

#Export to Excel
tabla_puntaje_mix.to_excel('./XLSX_files_output/N2_puntaje_categoria.xlsx',index=True)
tabla_puntaje_mix.head(10)


,NOMBRE CLIENTE,CANAL ACTUAL,MERCADO MO,CANAL MO,ETIQUETA TDN,score,puntaje_mix_2
Código Dynamics,,,,,,,
64,ACEITERA PALENCIA,0.0,TRADICIONAL,DETALLE,ACEITERA,5.00,A
990,MIGUEL LÓPEZ,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
52,LUBREMA S.A. DISTRIBUIDORA,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
1635,LUIS FERNANDO CHEN,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
234,IMPORTADORA Y EXPORTADORA CASTILLO,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
493,CELSO ROSBELI PEREZ CHAJ,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
23,REPUESTOS Y ACCESORIOS PARA MOTOS BARDALEZ,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
53,LUBRICANTES DE OCCIDENTE S A,0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A
2596,"INVERSIONES EN LUBRICANTES FT, S A",0.0,TRADICIONAL,DETALLE,ACEITERA,4.75,A


# B.5 Size

In [11]:
tabla_3 = read.drop_duplicates(subset=['Código Dynamics']).set_index("Código Dynamics")
tamano = tabla[["7.  CANTIDAD DE DEPENDIENTES DEL PDV", "9.  TAMAÑO DEL NEGOCIO ", "8.  BODEGA"]]

tamano = tamano.replace('1',0 ,regex=True)
tamano = tamano.replace('2',0 ,regex=True)
tamano = tamano.replace('3',1 ,regex=True)
tamano = tamano.replace('4',1 ,regex=True)
tamano = tamano.replace('5',1 ,regex=True)
tamano = tamano.replace('6',2 ,regex=True)
tamano = tamano.replace('7',2 ,regex=True)
tamano = tamano.replace('8',2 ,regex=True)
tamano = tamano.replace('9',3 ,regex=True)
tamano = tamano.replace('10',3 ,regex=True)
tamano = tamano.replace('MAS DE 10',3 ,regex=True)
tamano = tamano.replace('-',np.nan ,regex=True)
tamano = tamano.replace('Si',3 ,regex=True)
tamano = tamano.replace('No',0 ,regex=True)
tamano = tamano.replace('PEQUEÑO',0 ,regex=True)
tamano = tamano.replace('Mediano',1 ,regex=True)
tamano = tamano.replace('Grande',2 ,regex=True)
tamano = tamano.replace('EXTRA GRANDE',3 ,regex=True).fillna(0).astype('int32')


diccionario = {"7.  CANTIDAD DE DEPENDIENTES DEL PDV" : 0.22,
               "9.  TAMAÑO DEL NEGOCIO ":0.41,
               "8.  BODEGA":0.37 }


def puntajex(x):
    if x<1:
        return "C"
    elif x>=1 and x<2:
        return "B"
    elif x>=2:
        return "A"


tamano = tamano.mul(diccionario)
tamano["score"] = tamano.apply(lambda row: np.sum(row), axis=1) 
tamano["puntaje_tamaño"] =tamano["score"].apply(puntajex)

#MERGE CON CLIENTES
tamano_merge = pd.merge(tamano.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
tamano_merge =  tamano_merge[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","score","puntaje_tamaño"]].set_index('Código Dynamics')

#Export to Excel
tamano_merge.to_excel('./XLSX_files_output/N2_puntaje_tamaño.xlsx',index=True)


tamano_merge.head(10)


,NOMBRE CLIENTE,CANAL ACTUAL,MERCADO MO,CANAL MO,ETIQUETA TDN,score,puntaje_tamaño
Código Dynamics,,,,,,,
9419,PEDRO CRUZ,0.0,TRADICIONAL,DETALLE,MOSTRADOR REPUESTOS MOTOS,0.00,C
9416,DAYANA PAOLA PINEDA ZAMORA,0.0,TRADICIONAL,DETALLE,ACEITERA,0.22,C
9415,JOEL ALFREDO DE LEON TAQUE,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.00,C
9408,ELFIDO SALAZAR DE LA CRUZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.00,C
9406,ESTUARDO TECU GARCIA,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.00,C
9405,JANS KENNY LÓPEZ NORIEGA,0.0,TRADICIONAL,DETALLE,CONSUMO PROPIO,0.00,C
9398,"PESQUEROS DE GUATEMALA, S.A.",0.0,TRADICIONAL,DETALLE,SERVICIOS VARIOS,1.93,B
9394,VICTOR EMILIO IXCAJOC TICUN,0.0,TRADICIONAL,DETALLE,TALLER DE VEHICULO,0.00,C
9387,WILSON FRANKYN PEREZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,1.11,B


# B.6 Services

In [12]:
tabla_4 = read.drop_duplicates(subset=['Código Dynamics']).set_index("Código Dynamics")
servicios = tabla_4[["12.01 CAMBIO DE ACEITE",
                "12.05 MECANICA RAPIDA",
                "12.06 LAVADO/LIMPIEZA",
                "12.08 COLOCACION DE ACCESORIOS",
                "12.10 CAMBIO DE LLANTAS"]]


diccionario = {"12.01 CAMBIO DE ACEITE":0.40,
                "12.05 MECANICA RAPIDA":0.30,
                "12.06 LAVADO/LIMPIEZA":0.15,
                "12.08 COLOCACION DE ACCESORIOS":0.10,
                "12.10 CAMBIO DE LLANTAS":0.05}


def puntajex(x):
    if x<0.5:
        return "C"
    elif x>=0.5 and x<0.80:
        return "B"
    elif x>=0.80:
        return "A"


servicios = servicios.mul(diccionario)
servicios["score"] = servicios.apply(lambda row: np.sum(row), axis=1) 
servicios["puntaje_servicios"] =servicios["score"].apply(puntajex)

#MERGE CON CLIENTES
servicios_merge = pd.merge(servicios.reset_index(), clientes.reset_index(), how='inner', on=['Código Dynamics'])
servicios_merge =  servicios_merge[["Código Dynamics","NOMBRE CLIENTE ", "CANAL ACTUAL", "MERCADO MO ","CANAL MO ","ETIQUETA TDN ","score","puntaje_servicios"]]


#Export to Excel
servicios_merge.to_excel('./XLSX_files_output/N2_puntaje_servicios.xlsx',index=True)
servicios_merge.head(10)


,Código Dynamics,NOMBRE CLIENTE,CANAL ACTUAL,MERCADO MO,CANAL MO,ETIQUETA TDN,score,puntaje_servicios
0,9419,PEDRO CRUZ,0.0,TRADICIONAL,DETALLE,MOSTRADOR REPUESTOS MOTOS,0.40,C
1,9416,DAYANA PAOLA PINEDA ZAMORA,0.0,TRADICIONAL,DETALLE,ACEITERA,0.80,B
2,9415,JOEL ALFREDO DE LEON TAQUE,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.00,C
3,9408,ELFIDO SALAZAR DE LA CRUZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.70,B
4,9406,ESTUARDO TECU GARCIA,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.70,B
5,9405,JANS KENNY LÓPEZ NORIEGA,0.0,TRADICIONAL,DETALLE,CONSUMO PROPIO,0.00,C
6,9398,"PESQUEROS DE GUATEMALA, S.A.",0.0,TRADICIONAL,DETALLE,SERVICIOS VARIOS,0.00,C
7,9394,VICTOR EMILIO IXCAJOC TICUN,0.0,TRADICIONAL,DETALLE,TALLER DE VEHICULO,0.80,B
8,9387,WILSON FRANKYN PEREZ,0.0,TRADICIONAL,DETALLE,TALLER DE MOTOS,0.85,A
9,9383,CRISTIAN EDUARDO SANTOS IXCOT,0.0,TRADICIONAL,DETALLE,ACEITERA,0.80,B


# B.7 N2_Scoring

In [13]:
#Agregacion de puntajes
consolidado_N2 =pd.merge(servicios_merge, potencia_merge_[['puntaje_potencia']], on=['Código Dynamics'])
consolidado_N2 =pd.merge(consolidado_N2, inversion_merge[['puntaje_inversion']], on=['Código Dynamics'])
consolidado_N2 =pd.merge(consolidado_N2, tabla_puntaje_mix[['puntaje_mix_2']], on=['Código Dynamics'])
consolidado_N2 =pd.merge(consolidado_N2, tamano_merge[['puntaje_tamaño']], on=['Código Dynamics'])
consolidado_N2 = consolidado_N2.loc[:, consolidado_N2.columns != 'score'].set_index('Código Dynamics')


#Funciones
def normalizadorx(x):
    if x == "A":
        return 4
    if x == "B":
        return 3
    if x == "C":
        return 2
    if x == "D":
        return 1
    else:
        return x


#Definicion de ponderaciones y creacion de diccionario    
diccionario = {"puntaje_potencia" : 0.30,
               'puntaje_inversion':0.15,
               'puntaje_mix_2':0.25,
               'puntaje_servicios':0.05,
               'puntaje_tamaño': 0.25
                 }    
    
#Aplicacion de normalizadorx 
consolidado_2 = consolidado_N2.applymap(normalizadorx)


#Aplicacion de ponderaciones
for key, value in diccionario.items(): 
    consolidado_2[key] = consolidado_2[key] * value

#Suma de scores ponderados    
consolidado_2['score_final'] =round(consolidado_2.sum(axis=1))

#Aplicacion de condicionales para puntaje final
consolidado_2.loc[(consolidado_2['score_final'] <3 ), 'puntaje_final'] = "BAJO"
consolidado_2.loc[(consolidado_2['score_final'] >=3  ), 'puntaje_final'] = "MEDIO"
consolidado_2.loc[(consolidado_2['score_final'] >=3.5), 'puntaje_final'] = "ALTO"

consolidado_final = pd.merge(consolidado_2, consolidado_N2[['puntaje_potencia',"puntaje_servicios","puntaje_inversion","puntaje_mix_2","puntaje_tamaño"]], on=['Código Dynamics']).reset_index()
consolidado_final = consolidado_final[["Código Dynamics", "NOMBRE CLIENTE ","CANAL ACTUAL","CANAL MO ", "MERCADO MO ","ETIQUETA TDN ", "puntaje_potencia_x", "puntaje_potencia_y", "puntaje_inversion_x", "puntaje_inversion_y", "puntaje_mix_2_x", "puntaje_mix_2_y", "puntaje_servicios_x","puntaje_servicios_y","puntaje_tamaño_x","puntaje_tamaño_y", "puntaje_final"]]
consolidado_final = consolidado_final.drop_duplicates()

consolidado_final_0= consolidado_final.reset_index()
consolidado_final_0 = consolidado_final_0.astype({'Código Dynamics':'string'})
consolidado_final_0['Código Dynamics'] = consolidado_final_0['Código Dynamics'].apply(lambda x: x.zfill(6))

#Export to Excel
consolidado_final_0.to_excel('./XLSX_files_output/N2_PUNTAJE_FINAL.xlsx',index=False)
consolidado_final_0.head(10)


C:\Users\DELL\AppData\Local\Temp\ipykernel_1444\2621678728.py:40: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  consolidado_2['score_final'] =round(consolidado_2.sum(axis=1))


,index,Código Dynamics,NOMBRE CLIENTE,CANAL ACTUAL,CANAL MO,MERCADO MO,ETIQUETA TDN,puntaje_potencia_x,puntaje_potencia_y,puntaje_inversion_x,puntaje_inversion_y,puntaje_mix_2_x,puntaje_mix_2_y,puntaje_servicios_x,puntaje_servicios_y,puntaje_tamaño_x,puntaje_tamaño_y,puntaje_final
0,0,009419,PEDRO CRUZ,0.0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS MOTOS,0.6,C,0.30,C,0.50,C,0.10,C,0.50,C,BAJO
1,1,009416,DAYANA PAOLA PINEDA ZAMORA,0.0,DETALLE,TRADICIONAL,ACEITERA,0.9,B,0.45,B,0.75,B,0.15,B,0.50,C,MEDIO
2,2,009415,JOEL ALFREDO DE LEON TAQUE,0.0,DETALLE,TRADICIONAL,TALLER DE MOTOS,0.6,C,0.30,C,0.50,C,0.10,C,0.50,C,BAJO
3,3,009408,ELFIDO SALAZAR DE LA CRUZ,0.0,DETALLE,TRADICIONAL,TALLER DE MOTOS,0.9,B,0.30,C,1.00,A,0.15,B,0.50,C,MEDIO
4,4,009406,ESTUARDO TECU GARCIA,0.0,DETALLE,TRADICIONAL,TALLER DE MOTOS,0.6,C,0.30,C,0.75,B,0.15,B,0.50,C,BAJO
5,5,009405,JANS KENNY LÓPEZ NORIEGA,0.0,DETALLE,TRADICIONAL,CONSUMO PROPIO,0.6,C,0.30,C,0.50,C,0.10,C,0.50,C,BAJO
6,6,009398,"PESQUEROS DE GUATEMALA, S.A.",0.0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,0.6,C,0.30,C,0.75,B,0.10,C,0.75,B,BAJO
7,7,009394,VICTOR EMILIO IXCAJOC TICUN,0.0,DETALLE,TRADICIONAL,TALLER DE VEHICULO,0.9,B,0.30,C,0.75,B,0.15,B,0.50,C,MEDIO
8,8,009387,WILSON FRANKYN PEREZ,0.0,DETALLE,TRADICIONAL,TALLER DE MOTOS,0.6,C,0.60,A,0.75,B,0.20,A,0.75,B,MEDIO
9,9,009383,CRISTIAN EDUARDO SANTOS IXCOT,0.0,DETALLE,TRADICIONAL,ACEITERA,0.9,B,0.30,C,0.50,C,0.15,B,0.50,C,BAJO


# C. **MATRIX**

Finally, a matrix was generated with the results of N1 and N2, determining the optimal commercial strategy based on the segment in which each client falls. The matrix serves as a tool to guide decision-making regarding the commercial approach for each client segment. N1 and N2 represent key performance indicators or criteria that have been assessed and assigned scores for each client. These indicators could be metrics such as customer value, profitability, growth potential, or any other relevant factors based on the specific business context.

By mapping the N1 and N2 scores of each client onto the matrix, businesses can determine the appropriate commercial strategy for each segment. The matrix typically consists of different quadrants or sections that correspond to different strategies based on the combination of N1 and N2 scores.

In [14]:
import pandas as pd
import numpy as np

#Final N1
file = "./XLSX_files_output/N1_PUNTAJE_FINAL.xlsx"
read = pd.read_excel(file)
n1 = read.loc[:, read.columns != 'index'].set_index('Código Dynamics')

#Final N2
file = "./XLSX_files_output/N2_PUNTAJE_FINAL.xlsx"
read = pd.read_excel(file)
n2 = read.loc[:, read.columns != 'index'].set_index('Código Dynamics')


# Merge
consolidado =pd.merge(n1, n2[['puntaje_final']], how='left', on=['Código Dynamics'])
consolidado.rename(columns = {'puntaje_final_x':'Puntaje_N1', 'puntaje_final_y':'Puntaje_N2'}, inplace = True)
consolidado_final = consolidado[["NOMBRE CLIENTE ","CANAL ACTUAL","CANAL MO ", "MERCADO MO ","ETIQUETA TDN ",'Puntaje_N1','Puntaje_N2']].fillna("SIN_CENSO")
consolidado_final = consolidado_final.reset_index()

# Asignacion de Estrategia
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "DIAMANTE" ) & (consolidado_final['Puntaje_N2'] == "ALTO"),"Estrategia" ] = "BLINDAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "DIAMANTE" ) & (consolidado_final['Puntaje_N2'] == "MEDIO"),"Estrategia" ] = "BLINDAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "DIAMANTE" ) & (consolidado_final['Puntaje_N2'] == "BAJO"),"Estrategia" ] = "MANTENER"

consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "ORO" ) & (consolidado_final['Puntaje_N2'] == "ALTO"),"Estrategia" ] = "BLINDAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "ORO" ) & (consolidado_final['Puntaje_N2'] == "MEDIO"),"Estrategia" ] = "DESARROLLAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "ORO" ) & (consolidado_final['Puntaje_N2'] == "BAJO"),"Estrategia" ] = "MANTENER"

consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "PLATA" ) & (consolidado_final['Puntaje_N2'] == "ALTO"),"Estrategia" ] = "DESARROLLAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "PLATA" ) & (consolidado_final['Puntaje_N2'] == "MEDIO"),"Estrategia" ] = "DESARROLLAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "PLATA" ) & (consolidado_final['Puntaje_N2'] == "BAJO"),"Estrategia" ] = "OPTIMIZAR"

consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "BRONCE" ) & (consolidado_final['Puntaje_N2'] == "ALTO"),"Estrategia" ] = "DESARROLLAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "BRONCE" ) & (consolidado_final['Puntaje_N2'] == "MEDIO"),"Estrategia" ] = "DESARROLLAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == "BRONCE" ) & (consolidado_final['Puntaje_N2'] == "BAJO"),"Estrategia" ] = "OPTIMIZAR"

consolidado_final.loc[(consolidado_final['Puntaje_N1'] == 'DIAMANTE' ) & (consolidado_final['Puntaje_N2'] == 'SIN_CENSO'),"Estrategia" ] = "MANTENER"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == 'ORO' ) & (consolidado_final['Puntaje_N2'] == 'SIN_CENSO'),"Estrategia" ] = "MANTENER"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == 'PLATA' ) & (consolidado_final['Puntaje_N2'] == 'SIN_CENSO'),"Estrategia" ] = "OPTIMIZAR"
consolidado_final.loc[(consolidado_final['Puntaje_N1'] == 'BRONCE' ) & (consolidado_final['Puntaje_N2'] == 'SIN_CENSO'),"Estrategia" ] = "OPTIMIZAR"
consolidado_final = consolidado_final.drop_duplicates()

#Agregar ceros a código cliente
consolidado_final_0= consolidado_final.reset_index()
consolidado_final_0 = consolidado_final_0.astype({'Código Dynamics':'string'})
consolidado_final_0['Código Dynamics'] = consolidado_final_0['Código Dynamics'].apply(lambda x: x.zfill(6))

#Export to Excel
consolidado_final_0.to_excel('./XLSX_files_output/N2_N1_PUNTAJES_ESTRATEGIAS.xlsx',index=False)
consolidado_final_0.head(10)



,index,Código Dynamics,NOMBRE CLIENTE,CANAL ACTUAL,CANAL MO,MERCADO MO,ETIQUETA TDN,Puntaje_N1,Puntaje_N2,Estrategia
0,0,000226,GT TRAIDING,0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,ORO,BAJO,MANTENER
1,1,001413,MOVIMIENTO DE MAQUINARIA COPROPIEDAD,0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,ORO,BAJO,MANTENER
2,2,000018,LUBRICANTES GARCIA II,0,DETALLE,TRADICIONAL,ACEITERA,DIAMANTE,ALTO,BLINDAR
3,3,001106,INTERSUM S.A.,0,DETALLE,TRADICIONAL,SERVICIOS VARIOS,ORO,MEDIO,DESARROLLAR
4,4,001207,"DISTRIBUIDORA AP&P, SOCIEDAD ANONIMA",0,DETALLE,TRADICIONAL,FERRETERIA,ORO,MEDIO,DESARROLLAR
5,5,000031,REPUESTOS MOLINA,0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,DIAMANTE,MEDIO,BLINDAR
6,6,000176,REPUESTOS KAIROS,0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,DIAMANTE,MEDIO,BLINDAR
7,7,000090,REPUESTOS DE LEON,0,DETALLE,TRADICIONAL,ACEITERA,DIAMANTE,MEDIO,BLINDAR
8,8,000029,"TXC, S.A.",0,DETALLE,TRADICIONAL,ACEITERA,DIAMANTE,ALTO,BLINDAR
9,9,000118,REPUESTOS AUOTOTOTAL,0,DETALLE,TRADICIONAL,MOSTRADOR REPUESTOS VEHICULOS,DIAMANTE,ALTO,BLINDAR
